# GCP Weather Function

---
## Install functions-framework 💾
If you haven't already installed functions-framework, you will need to for testing the cloud function locally. Uncomment the code below, install, and then recomment the code - you only need to install it once.

In [40]:
# !pip install functions-framework

## Definition of Weather Function

In [17]:
import functions_framework
import pandas as pd
from sqlalchemy import create_engine, text
from datetime import datetime, timedelta
import requests
from dotenv import load_dotenv
import os

@functions_framework.http
def GetWeatherInfo (request):
    
    # Connection setup
    load_dotenv("variables.env")
    schema = os.getenv("schema")
    host = os.getenv("host")
    user = os.getenv("user")
    password = os.getenv("password")
    port = os.getenv("port")
    
    # Create connection string
    connection_string = create_engine(f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}')

    # get cities from database
    cities_df = pd.read_sql("city_names", con=connection_string)
    cities = list(cities_df['City'])
    
    # get geo info from SQL:
    geo_df = pd.read_sql("geo", con=connection_string)
    
    # get city IDs from SQL:
    city_df = pd.read_sql("city_names", con=connection_string)
    city_ids =[]
    for city in cities:
        if city in city_df['City'].values:
            city_ids.append(city_df.loc[city_df['City'] == city, 'city_id'].iloc[0])
        else:
            return print(f'{city} is not in the data base and has to be added first.')

    # get current weather info in data base from SQL:
    SQL_weather_df = pd.read_sql("weather", con=connection_string)
    
    # get wehter data from openweathermap:
    cities_weather_json =[]
    for city_id in city_ids:
            
        # Define the parameters to be passed into the url.
        weather_API_key = os.getenv("weather_API_key")
        longitude = geo_df.loc[geo_df['city_id'] == city_id, 'Longitude'].values[0]
        latitude = geo_df.loc[geo_df['city_id'] == city_id, 'Latitude'].values[0]
        
        # Reference the parameters in the url.
        URL = f"https://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&appid={weather_API_key}&units=metric"
        response = requests.get(URL)
        cities_weather_json.append(response.json())
    
     # extract valuable weather data into a data frame: 
    columns = [
        'temperature_deg_C', 'temp_min_deg_C', 'temp_max_deg_C', 'pressure_in_hPa', 'humidity_in_percent',
        'weather_info', 'weather_desc', 'wind_speed_in_m_per_sec',
        'wind_direction_in_deg', 'rain_probability', 'rain_last_3h_in_mm', 'timestamp_data_retrieved'
    ]
    
    weather_dfs = pd.DataFrame(columns=columns)
    city_count = 0
    
    for json in cities_weather_json:
        # Extract relevant weather information
        weather_df = pd.DataFrame(json['list'])
        weather_df['temperature_deg_C'] = weather_df.apply(lambda row: row['main']['temp'], axis=1)
        weather_df['temp_min_deg_C'] = weather_df.apply(lambda row: row['main']['temp_min'], axis=1)
        weather_df['temp_max_deg_C'] = weather_df.apply(lambda row: row['main']['temp_max'], axis=1)
        weather_df['pressure_in_hPa'] = weather_df.apply(lambda row: row['main']['grnd_level'], axis=1)
        weather_df['humidity_in_percent'] = weather_df.apply(lambda row: row['main']['humidity'], axis=1)
        weather_df['weather_info'] = weather_df.apply(lambda row: row['weather'][0]['main'], axis=1)
        weather_df['weather_desc'] = weather_df.apply(lambda row: row['weather'][0]['description'], axis=1)
        weather_df['wind_speed_in_m_per_sec'] = weather_df.apply(lambda row: row['wind']['speed'], axis=1)
        weather_df['wind_direction_in_deg'] = weather_df.apply(lambda row: row['wind']['deg'], axis=1)
        weather_df['rain_probability'] = weather_df.apply(lambda row: row['pop'], axis=1)
        weather_df['rain_last_3h_in_mm'] = weather_df.apply(lambda row: row['rain'].get('3h', 0) if 'rain' in row and isinstance(row['rain'], dict) else 0, axis=1)
        weather_df['city_id'] = city_ids[city_count]
        weather_df['timestamp_data_retrieved'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        weather_df = weather_df[['dt_txt', 'temperature_deg_C', 'temp_min_deg_C', 'temp_max_deg_C', 'pressure_in_hPa', 'humidity_in_percent', 
                                 'weather_info', 'weather_desc', 'wind_speed_in_m_per_sec', 
                                 'wind_direction_in_deg', 'rain_probability', 'rain_last_3h_in_mm', 'city_id', 'timestamp_data_retrieved']]
        
        city_count += 1
    
        # Concatenate the data
        weather_dfs = pd.concat([weather_dfs, weather_df], ignore_index=True)
        
    print(weather_dfs.dtypes)
    print(weather_dfs.head())
    print(weather_dfs.isnull().sum())

    weather_dfs['city_id'] = pd.to_numeric(weather_dfs['city_id'], errors='coerce').fillna(0).astype(int)
    weather_dfs['pressure_in_hPa'] = pd.to_numeric(weather_dfs['pressure_in_hPa'], errors='coerce').fillna(0).astype(int)
    weather_dfs['humidity_in_percent'] = pd.to_numeric(weather_dfs['humidity_in_percent'], errors='coerce').fillna(0).astype(int)
    weather_dfs['wind_direction_in_deg'] = pd.to_numeric(weather_dfs['wind_direction_in_deg'], errors='coerce').fillna(0).astype(int)
    weather_dfs['rain_last_3h_in_mm'] = pd.to_numeric(weather_dfs['rain_last_3h_in_mm'], errors='coerce').fillna(0.0).astype(float)
    weather_dfs['dt_txt'] = pd.to_datetime(weather_dfs['dt_txt'])

    # Create the list of tuples for deletion
    duplicate_weather_df = SQL_weather_df.merge(weather_dfs[['city_id', 'dt_txt']], on=['city_id', 'dt_txt'], how='inner')[['city_id', 'dt_txt']]
    if len(duplicate_weather_df) == 0:
        pass
    else:
        delete_combos_list = [(duplicate_weather_df['city_id'][i], str(duplicate_weather_df['dt_txt'][i])) for i in range(len(duplicate_weather_df))]
        
        # Build placeholders( values will be assigend by the params dict in the execute function)
        placeholders = ', '.join([f"(:city_id{i}, :dt_txt{i})" for i in range(len(delete_combos_list))])
        
        # Build the DELETE query
        delete_query = f"""
        DELETE FROM weather
        WHERE (city_id, dt_txt) IN ({placeholders});
        """
        
        # Build the parameter dictionary for SQL execution
        params = {}
        for i, (cid, dt) in enumerate(delete_combos_list):
            params[f'city_id{i}'] = cid
            params[f'dt_txt{i}'] = dt
        
        # Execute the query
        with connection_string.connect() as conn:
            conn.execute(text(delete_query), params)
            conn.commit()
    
    weather_dfs.to_sql('weather',
                  if_exists='append',
                  con=connection_string,
                  index=False)
    
    return "Data has been sent to the table"

## Test Weather Function

In [19]:
from flask import Request
import json

request_data = {}
request = Request.from_values(data=json.dumps(request_data))

# Call the function
response = GetWeatherInfo(request)
print(response)

C:\Users\wulko\AppData\Local\Temp\ipykernel_37044\3623104856.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  weather_dfs = pd.concat([weather_dfs, weather_df], ignore_index=True)


temperature_deg_C           float64
temp_min_deg_C              float64
temp_max_deg_C              float64
pressure_in_hPa              object
humidity_in_percent          object
weather_info                 object
weather_desc                 object
wind_speed_in_m_per_sec     float64
wind_direction_in_deg        object
rain_probability            float64
rain_last_3h_in_mm          float64
timestamp_data_retrieved     object
dt_txt                       object
city_id                     float64
dtype: object
   temperature_deg_C  temp_min_deg_C  temp_max_deg_C pressure_in_hPa  \
0              22.91           22.91           24.53            1006   
1              22.87           22.79           22.87            1006   
2              21.77           21.20           21.77            1007   
3              20.62           20.62           20.62            1007   
4              19.61           19.61           19.61            1007   

  humidity_in_percent weather_info      weather_d